<a href="https://colab.research.google.com/github/princedl/dl4j-colab/blob/master/dl4j-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Run deeplearning4j on Google colab

This notebook runs the dl4j-examples on google colab. Execute the section *Access the dl4j-ui via a SSH tunnel* if you need access to the dl4j web-ui's.


NOTE: This notebook builds only the dl4j-examples module by default, change ```MODULE``` below to build another module.




In [0]:
MODULE="dl4j-examples"

Install maven & JAVA 8

In [0]:
!apt-get install maven
!apt-get install openjdk-8-jdk
!apt-get install openjfx

Run the cell below and activate JAVA 8

In [0]:
!update-alternatives --config java


Clone the dl4j-examples repository


In [1]:
!git clone https://github.com/deeplearning4j/dl4j-examples.git

Cloning into 'dl4j-examples'...
remote: Enumerating objects: 35305, done.
remote: Total 35305 (delta 0), reused 0 (delta 0), pack-reused 35305
Receiving objects: 100% (35305/35305), 344.08 MiB | 30.77 MiB/s, done.
Resolving deltas: 100% (7633/7633), done.
Checking out files: 100% (15273/15273), done.


Ammend dl4j-examples.

In [4]:
%cat dl4j-examples/runexamples.sh

#!/usr/bin/env bash
# Run one or all of the examples.
# Use runexamples.sh -h to see the options.

help() {
  [ -n "$1" ] && echo $1
  cat <<-EOF1
usage: $0 [-h|--help] [-a|-all] [-n|--no-pauses]
where:
  -h|--help          Show help and quit
  -a|--all           Run all the examples. Default is to prompt for which one to run.
  -n|--no-pauses     Don't pause between examples (use with --all).
}
EOF1
}

# This is shown only if all examples are executed.
banner() {
  if [ $all -eq 0 ]
  then
    cat <<-EOF2

    deeplearning4j examples:

    Each example will be executed, then some of them will pop up a
    dialog with a data plot. Quit the data plot application to proceed
    to the next example.
EOF2
    if [ $pauses -eq 0 ]
    then
    cat <<-EOF2

    We'll pause after each one; hit <return> to continue or <ctrl-c>
    to quit.
EOF2
    fi
    cat <<-EOF3

EOF3
  fi
}


let all=1
let pauses=0
while [ $# -ne 0 ]
do
  case $1 in
    -h|--h*)
      help
      exit 0
      ;;
    -a|--

In [5]:
!sed -i -e 's/java -cp/java -Dorg.deeplearning4j.ui.port=9001 -cp/' dl4j-examples/runexamples.sh

#!/usr/bin/env bash
# Run one or all of the examples.
# Use runexamples.sh -h to see the options.

help() {
  [ -n "$1" ] && echo $1
  cat <<-EOF1
usage: $0 [-h|--help] [-a|-all] [-n|--no-pauses]
where:
  -h|--help          Show help and quit
  -a|--all           Run all the examples. Default is to prompt for which one to run.
  -n|--no-pauses     Don't pause between examples (use with --all).
}
EOF1
}

# This is shown only if all examples are executed.
banner() {
  if [ $all -eq 0 ]
  then
    cat <<-EOF2

    deeplearning4j examples:

    Each example will be executed, then some of them will pop up a
    dialog with a data plot. Quit the data plot application to proceed
    to the next example.
EOF2
    if [ $pauses -eq 0 ]
    then
    cat <<-EOF2

    We'll pause after each one; hit <return> to continue or <ctrl-c>
    to quit.
EOF2
    fi
    cat <<-EOF3

EOF3
  fi
}


let all=1
let pauses=0
while [ $# -ne 0 ]
do
  case $1 in
    -h|--h*)
      help
      exit 0
      ;;
    -a|--

Move to the directory of the module to build

In [0]:
%cd /content/dl4j-examples/{MODULE}
!ls -al

Compile the examples

NOTE: This runs for a while when executed for the first time as it downloads all the dependencies. As it generates a lot of output we pipe it into a file and check afterwards if all went well. This takes up to 2 minutes for the ```dl4j-examples``` module.

In [0]:
!mvn compile &>compile.log

In [0]:
!tail -30 compile.log

Package the examples

NOTE: This will create an Uber-jar which can take up to 5 minutes for the ```dl4j-examples``` module.

In [0]:
!mvn package 

List all examples (Code is taken from ```dl4j-examples/runexamples.sh```)

In [0]:
!cd ./src/main/java/ && find . -name '*.java' -exec grep -l 'void main' {} \; | sed "s?/?.?g" | sed "s?^\.*\(.*\)\.java?\1?"

Set ```CLASS_TO_RUN``` to the example main class name to execute it.

In [0]:
CLASS_TO_RUN = "org.deeplearning4j.examples.arbiter.BasicHyperparameterOptimizationExample" 
#Run dl4j-web-guis on port 9001 to avoid a port conflict on colab
!mvn exec:java -Dexec.mainClass={CLASS_TO_RUN} -Dorg.deeplearning4j.ui.port=9001 

##Access the dl4j-ui via a SSH tunnel

To access the dl4j-ui you need to establish an SSH tunnel from the colab-server to a machine with a public IP.


Find your public IP
https://whatismyipaddress.com/


dl4j web-ui's:


* TRAIN  | http://localhost:9000/train |
* ARBITER |  http://localhost:9000/arbiter |




In [0]:
#USER/HOST will be used to open a reverse ssh-tunnel to HOST 
#so that the dl4j-web-guis cn be accessed from that machine

USER = 'pdl'
HOST = '84.72.80.86'

USER_AT_HOST = USER+"@"+HOST

print(USER_AT_HOST)

#PASSWORD is needed to ssh-copy the public key to USER_AT_HOST
PASSWORD = 'ENTER PASSWORD FOR USER_AT_HOST_HERE'

Generate and share a SSH-Key with HOST - this will be used to access the dl4j web-guis remotelly.

In [0]:
!apt-get install sshpass
!apt-get install autossh

In [0]:
!ssh-keygen -C `hostname`

In [0]:
!sshpass -p{PASSWORD} ssh-copy-id -o StrictHostKeyChecking=no {USER_AT_HOST}

In [0]:
#Run this to check the ssh connection
#!ssh -o StrictHostKeyChecking=no {USER_AT_HOST}

Open an reverse ssh tunnel from HOST to this colab machine to access the dl4j web gui. Click here to open it https://localhost:9000.

In [0]:
%%bash -s "$USER_AT_HOST" --bg --proc bg_proc
autossh -vnN -o StrictHostKeyChecking=no -R 9000:localhost:9001 $1 &>file

In [0]:
#Use this to check if everything is OK
print (bg_proc.pid)

%cat file

In [0]:
!ps -al {bg_proc.pid}


In [0]:
#Use this to kill the tunnel process
#!kill {{bg_proc.pid}